In [74]:
import time
import asyncio
import nest_asyncio
import traceback
import os
import torch
from dataclasses import dataclass, field
from typing import List, Dict, Any, Optional
from dotenv import load_dotenv
from langchain_core.messages import HumanMessage, AIMessage, ToolMessage, SystemMessage
from langchain_core.output_parsers import JsonOutputParser

# LangChain Local Providers
# from langchain_community.llms import Ollama
from langchain_community.chat_models import ChatOllama

from langchain.chat_models import init_chat_model

import json
import re


In [75]:
# This is required to run LangGraph's async loops in a Jupyter Notebook
nest_asyncio.apply()

In [144]:
from stockanalyzer.crew import SUPERVISOR_PROMPT, WORKER_PROMPT, SYNTHESIS_PROMPT
from stockanalyzer.crew import AgentState, ContextSchema
from stockanalyzer.crew import SupervisorPlan, create_worker_node

from stockanalyzer.config import QWEN_3, GEMINI_2_5_FLASH
from stockanalyzer.config import Config, ModelProvider, ModelConfig

from stockanalyzer.tools import get_historical_stock_price, fetch_sec_filing_sections, Section 

from edgar import set_identity
set_identity("abc@123.com")

In [103]:
Config.MODEL = ModelConfig("qwen2.5:1.5b", temperature=0.0, provider=ModelProvider.OLLAMA)
Config.MODEL

ModelConfig(name='qwen2.5:1.5b', temperature=0.0, provider=<ModelProvider.OLLAMA: 'ollama'>)

In [97]:
#Copied from main.py
def create_model():
    parameters = {
        "temperature": Config.MODEL.temperature,
        "thinking_budget": 0
    }
    if Config.MODEL.provider == ModelProvider.OLLAMA:
        parameters["num_ctx"] = Config.CONTEXT_WINDOW
        
    return init_chat_model(
        f"{Config.MODEL.provider.value}:{Config.MODEL.name}",
        **parameters
    )

In [104]:
model = create_model()
print(f'model type: {type(model)}')
print(f'model: {model}')

model type: <class 'langchain_community.chat_models.ollama.ChatOllama'>
model: model='qwen2.5:1.5b' num_ctx=8192 temperature=0.0


In [99]:
pricing_analyst = create_worker_node("Price Analyst", [get_historical_stock_price])
pricing_analyst

<function stockanalyzer.crew.create_worker_node.<locals>.worker_node(state: stockanalyzer.crew.AgentState, runtime: langgraph.runtime.Runtime[stockanalyzer.crew.ContextSchema])>

In [100]:
# Create empty instance of AgentState
state = AgentState(
        messages = [HumanMessage(content="Analyze NVDIA stock performance based on historical data."
                                  , name="SupervisorInstruction")],
        next_agent = "Pricing Analyst",
        iteration_count = 1
    )
state

AgentState(messages=[HumanMessage(content='Analyze NVDIA stock performance based on historical data.', additional_kwargs={}, response_metadata={}, name='SupervisorInstruction')], iteration_count=1, next_agent='Pricing Analyst')

In [101]:
#Example tool call
def tool_call(agent_name: str, state: AgentState, tools: list, model):
    """
    A legacy-compatible tool call function that doesn't rely on .bind_tools().
    Works with langchain-core < 0.3.0 and ChatOllama.
    """
    
    # 1. Manually construct the tool definitions for the prompt
    tool_defs = []
    for t in tools:
        tool_defs.append({
            "name": t.name,
            "description": t.description,
            "parameters": t.args if hasattr(t, 'args') else "Expects a ticker string"
        })

    # 2. Build a strict system instruction for JSON output
    system_instruction = (
        f"You are the {agent_name}. Your task is to process instructions by choosing a tool.\n"
        f"AVAILABLE TOOLS:\n{json.dumps(tool_defs, indent=2)}\n\n"
        f"INSTRUCTION: {state.messages[-1].content}\n\n"
        f"You MUST respond ONLY with a JSON object in this format:\n"
        f"{{\"tool_name\": \"name_of_tool\", \"args\": {{\"param_name\": \"value\"}}}}\n"
        f"Do not include any other text or explanations."
    )
    
    # 3. Invoke the model
    # Note: Using invoke() which is supported in your version, but without tool binding
    response = model.invoke([HumanMessage(content=system_instruction)])
    
    try:
        # 4. Clean and parse the response
        content = response.content
        # Find the JSON block if the model added markdown backticks
        json_match = re.search(r"\{.*\}", content, re.DOTALL)
        if json_match:
            data = json.loads(json_match.group(0))
            
            # 5. Manually construct an AIMessage with tool_calls for downstream compatibility
            # In older LangChain, we simulate the tool_call structure
            return AIMessage(
                content="",
                additional_kwargs={
                    "tool_calls": [{
                        "id": "call_" + str(hash(content)),
                        "function": {
                            "name": data["tool_name"],
                            "arguments": json.dumps(data["args"])
                        },
                        "type": "function"
                    }]
                }
            )
    except Exception as e:
        # Return a fallback message if parsing fails
        return AIMessage(content=f"Error parsing tool call: {str(e)}")

# Usage example in your notebook:
# response = tool_call("Pricing Analyst", state, [get_historical_stock_price], model)

In [138]:
# Run the tool call
tool_call_response_price_analyst = tool_call("Pricing Analyst", state, [get_historical_stock_price], model)
tool_call_response_filing_analyst = tool_call("Filing Analyst", state, [fetch_sec_filing_sections], model)


In [139]:
tool_call_response_price_analyst.tool_calls


[{'name': 'get_historical_stock_price',
  'args': {'ticker': 'NVDA'},
  'id': 'call_4389698801266433013',
  'type': 'tool_call'}]

In [159]:
tool_call_response_filing_analyst.tool_calls[0]

{'name': 'fetch_sec_filing_sections',
 'args': {'ticker': 'NVDA', 'sections': ['income_statement']},
 'id': 'call_-3450241622400116513',
 'type': 'tool_call'}

In [ ]:
tool_output_price = get_historical_stock_price.invoke(tool_call_response_price_analyst.tool_calls[0]["args"])
# tool_output = get_historical_stock_price.func(tool_call_response_price_analyst.tool_calls[0]["args"]["ticker"])

In [ ]:
tool_output_filing = fetch_sec_filing_sections.invoke(tool_call_response_filing_analyst.tool_calls[0]["args"])
len(tool_output_filing)

7460

In [161]:
tool_messages = [ToolMessage(content=str(tool_output_filing), 
                            tool_call_id=tool_call_response_filing_analyst.tool_calls[0]['id'],
                            name=tool_call_response_filing_analyst.tool_calls[0]["name"])]

In [162]:
summary_response = model.invoke(
    WORKER_PROMPT.format(
        agent_name="Filing Analyst",
        supervisor_instruction="Analyze NVDIA stock performance based on historical data.",
        tool_data=tool_messages[0].content # Only one tool call (so far)
    )
)

In [168]:
summary_response

AIMessage(content='NVIDIA CORP Stock Performance Analysis\n\nKey Findings:\n\n1. **Revenue Growth**: The company reported a significant increase in revenue across all segments, with Compute & Networking and Graphics experiencing substantial growth.\n2. **Operating Income**: Operating income increased by 57% year-over-year, driven primarily by higher gross profit margins and reduced operating expenses.\n3. **Net Income**: Net income also saw an impressive rise of 109%, reflecting improved profitability across all segments.\n\nThese findings indicate a strong financial performance with robust growth in revenue and net income, particularly in key segments like Compute & Networking and Graphics.', additional_kwargs={}, response_metadata={'model': 'qwen2.5:1.5b', 'created_at': '2026-01-01T02:45:10.7154849Z', 'message': {'role': 'assistant', 'content': ''}, 'done': True, 'done_reason': 'stop', 'total_duration': 85340411700, 'load_duration': 2111083600, 'prompt_eval_count': 1712, 'prompt_eval

In [165]:
print(summary_response.content)

NVIDIA CORP Stock Performance Analysis

Key Findings:

1. **Revenue Growth**: The company reported a significant increase in revenue across all segments, with Compute & Networking and Graphics experiencing substantial growth.
2. **Operating Income**: Operating income increased by 57% year-over-year, driven primarily by higher gross profit margins and reduced operating expenses.
3. **Net Income**: Net income also saw an impressive rise of 109%, reflecting improved profitability across all segments.

These findings indicate a strong financial performance with robust growth in revenue and net income, particularly in key segments like Compute & Networking and Graphics.
